In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.optim as optim
import matplotlib.pyplot as plt
import sys, os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)

from Resources.Model import Model_v4
from Resources.Game import *


In [8]:
gamma = 0.98

error_hist = torch.load('./error_hist QL v4')
game_count = 0
draw_count = 0

criterion = nn.MSELoss()
model = Model_v4() # Conv large
model.train()
model.load_state_dict(torch.load('./Model Saves QL v4/model_1510_games'))
white_wins = 0
black_wins = 0
draws = 0

In [12]:
i_limit = 1000
while True:
    t0 = time.time()

    game = Game()
    next_root = None
    i = 0
    boards_white = [];  boards_black = []
    mat_white = [];     mat_black = []
    value_white = [];   value_black = []

    current_boards = []; current_labels = []

    model.eval()

    while not game.is_over():
        i += 1
        if i > i_limit:
            break

        moves = game.PossibleMoves()

        game_ini = game.copy()
        current_boards.append(board_to_tensor(game.pieces))
        board_batch = [board_to_tensor(game.pieces)]

        mate = False

        for move in moves:
            game.PlayMove(move)
            board_batch.append(board_to_tensor(game.pieces))
            game.FlipBoard()
            if game.is_over():
                mate = True
                chosen_move = move
                game = game_ini.copy()
                break
            game = game_ini.copy()


        board_tensor = torch.stack(board_batch)
        values = model(board_tensor)

        if i > 2: # both sides played one move, start filling up labels
            current_labels.append( max([values[i] for i in range(1, len(values))]) )

        if not mate:

            values_diff = [values[i] for i in range(1, len(values))]
            move_prob = torch.softmax(torch.Tensor(values_diff), dim=0).numpy()
            chosen_i = np.random.choice(range(len(moves)), p=move_prob)
            chosen_move = moves[chosen_i]
            
        game.PlayMove(chosen_move)

        mat_diff = game.MaterialDiff()

        if i % 2 == 1:
            #boards_white.append(board_to_tensor(game.pieces))
            mat_white.append(mat_diff)
            value_white.append(values[0].detach().item())
        if i % 2 == 0:
            #boards_black.append(board_to_tensor(game.pieces))
            mat_black.append(mat_diff)
            value_black.append(values[0].detach().item())

        game.FlipBoard()

    # plt.plot(model(torch.stack(boards_white)).detach().numpy())
    # plt.title('values white')
    # plt.show()
    # plt.plot(model(torch.stack(boards_black)).detach().numpy())
    # plt.title('values black')
    # plt.show()

    current_labels.append(-1)
    current_labels.append(1)

    if i <= i_limit: # game actually ended

        winner = game.get_winner()
        if winner == 'draw':
            draws += 1;         reward_white = 0;   reward_black = 0
        if winner == 'white':
            white_wins += 1;    reward_white = 1;   reward_black = -1
        if winner == 'black':
            black_wins += 1;    reward_white = -1;  reward_black = 1

    if i > i_limit:
        print('game termianted')
        continue

    game_time = time.time() - t0

    inputs_tens = torch.stack(current_boards)
    labels_tens = torch.Tensor(current_labels)   

    if winner != 'draw':
        # newest training error
        out = model(inputs_tens)
        out = out.view(out.shape[0])
        loss = criterion(out, labels_tens)
        error_hist.append(loss.item())
    else:
        draw_count += 1
        continue

    if game_count % 10 == 0:
        print(' -- {} -- winner: {}, i: {}   '.format(game_count, winner, i), 'wins: w = {}, b = {}, d = {}'.format(white_wins, black_wins, draws))


    if game_count % 100 == 1:# or True:
        plt.figure(figsize=(6, 3))
        plt.plot(mat_white, label='mat white')
        plt.legend()
        plt.show()
        plt.figure(figsize=(6, 3))
        plt.plot(value_white, label='value white')
        plt.plot(value_black, label='value black')
        plt.legend()
        plt.show()

    torch.save(inputs_tens, './Game Saves QL v4/inputs_{}'.format(game_count))
    torch.save(labels_tens, './Game Saves QL v4/labels_{}'.format(game_count))

    t1 = time.time()

    if game_count > 5:
        # sample some of previous game for replay. More recent -> more likely
        decay_rate = 0.003
        prob =  np.exp(-decay_rate * np.arange(game_count)) 
        prob = prob /sum(prob)
        num_samples = 3
        samples = np.random.choice(np.arange(game_count, 0, -1), size=num_samples, p=prob)

        for indices in samples:
            inputs_load = torch.load('./Game Saves QL v4/inputs_{}'.format(indices))
            inputs_tens = torch.cat((inputs_load, inputs_tens))
            labels_tens = torch.cat((torch.load('./Game Saves QL v4/labels_{}'.format(indices)), labels_tens))

    len_boards = len(inputs_tens)
    for c in range(len_boards):
        tens = inputs_tens[c]
        # if no pawns are on the board: add all other 3 rotated versions of the board to the current data set
        if torch.sum(inputs_tens[c][0]) + torch.sum(inputs_tens[c][6]) == 0:
            board_new = tensor_to_board(tens)
            for _ in range(3):
                board_new = rotate_board(board_new, 1)
                tens_new = torch.stack([board_to_tensor(board_new)])
                inputs_tens = torch.cat((inputs_tens, tens_new), 0)
                labels_tens = torch.cat((labels_tens, torch.stack([labels_tens[c]])), 0)
    
    model.train()
    # learning_rate = 5e-4 * 500 / (500 + game_count) # reducing learning rate, 1/n one possible options
    learning_rate = 1e-5
    # weight_decay = 1e-2 # regularization to avoid overfitting
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    random_order = [i for i in range(len(inputs_tens))]
    np.random.shuffle(random_order)
    for c in random_order:

        inp = torch.stack([inputs_tens[c], mirror_board_tensor(inputs_tens[c])])
        label = torch.stack([labels_tens[c], labels_tens[c]])

        out = model(inp)
        out = out.view(out.shape[0])

        loss = criterion(out, label)
        optimizer.zero_grad()
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step()

    torch.save(error_hist, './error_hist QL v4')

    if game_count % 100  == 1:
        interval = 50
        plt.figure()
        plt.plot([np.mean(error_hist[i:i+interval]) for i in range(len(error_hist) - interval)])
        plt.show()

    if game_count % 100 == 9:    
        torch.save(model.state_dict(), './Model Saves QL v4/model_{}_games'.format(game_count+1))

    game_count += 1


KeyboardInterrupt: 